# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Import PPrint
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file_path = "../WeatherPy/output_data/cities.csv"
city_data = pd.read_csv(data_file_path)

# Drop column that has a duplicate index
city_data = city_data.iloc[:, 1:9]
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Butaritari,3.0707,172.7902,81.23,77,44,8.99,KI
1,Hithadhoo,-0.6000,73.0833,82.94,74,82,12.30,MV
2,Wladyslawowo,54.7909,18.4009,30.00,68,0,14.97,PL
3,Khatanga,71.9667,102.5000,-29.67,85,82,9.33,RU
4,Cockburn Town,21.4612,-71.1419,78.21,77,0,19.53,TC


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

# Create base map
fig = gmaps.figure(center = (0, 0), zoom_level = 2)

# Extract location coords and humidity

locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)

# Create a heatmap layer

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 1)

# Add to map

fig.add_layer(heat_layer)

# Display map

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down cities to fit weather conditions

hotel_df = city_data.loc[(city_data["Max Temp"] >= 70) & 
                         (city_data["Max Temp"] < 80) & 
                         (city_data["Wind Speed"] < 10) & 
                         (city_data["Cloudiness"] == 0)]

# Reset index
hotel_df = hotel_df.reset_index(drop = True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Hermanus,-34.4187,19.2345,75.99,75,0,5.01,ZA
1,Port Hedland,-20.3167,118.5667,77.00,83,0,6.91,AU
2,Vao,-22.6667,167.4833,78.57,78,0,5.10,NC
3,Tra Vinh,9.9347,106.3453,71.02,83,0,8.01,VN
4,Lunavada,23.1333,73.6167,73.40,23,0,5.75,IN
5,Salalah,17.0151,54.0924,75.20,69,0,2.30,OM
6,Isangel,-19.5500,169.2667,77.43,81,0,3.40,VU
7,Voh,-20.9667,164.7000,72.99,86,0,2.55,NC
8,Lima,-12.0432,-77.0282,78.80,61,0,9.22,PE
9,Luxor,25.6989,32.6421,77.00,31,0,9.28,EG


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create an empty column named "Hotel Name"

hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Hermanus,-34.4187,19.2345,75.99,75,0,5.01,ZA,
1,Port Hedland,-20.3167,118.5667,77.00,83,0,6.91,AU,
2,Vao,-22.6667,167.4833,78.57,78,0,5.10,NC,
3,Tra Vinh,9.9347,106.3453,71.02,83,0,8.01,VN,
4,Lunavada,23.1333,73.6167,73.40,23,0,5.75,IN,
5,Salalah,17.0151,54.0924,75.20,69,0,2.30,OM,
6,Isangel,-19.5500,169.2667,77.43,81,0,3.40,VU,
7,Voh,-20.9667,164.7000,72.99,86,0,2.55,NC,
8,Lima,-12.0432,-77.0282,78.80,61,0,9.22,PE,
9,Luxor,25.6989,32.6421,77.00,31,0,9.28,EG,


In [6]:

# Create a params dictionary

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": "",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Retrieve closest hotel to location in coords, using a for loop

for index, row in hotel_df.iterrows():
    
    # Get the city and coords
    city_to_query = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the city to lookup
    params["location"] = f"{lat}, {lng}"
    
    # Assemble URL and make API request
    print(f"Retrieving Results for City #{index}: {city_to_query}")
    response = requests.get(base_url, params = params).json()

    results = response['results']
    
    hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
hotel_df

Retrieving Results for City #0: Hermanus
Retrieving Results for City #1: Port Hedland
Retrieving Results for City #2: Vao
Retrieving Results for City #3: Tra Vinh
Retrieving Results for City #4: Lunavada
Retrieving Results for City #5: Salalah
Retrieving Results for City #6: Isangel
Retrieving Results for City #7: Voh
Retrieving Results for City #8: Lima
Retrieving Results for City #9: Luxor
Retrieving Results for City #10: Mongo


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Hermanus,-34.4187,19.2345,75.99,75,0,5.01,ZA,Misty Waves Boutique Hotel
1,Port Hedland,-20.3167,118.5667,77.00,83,0,6.91,AU,The Esplanade Hotel
2,Vao,-22.6667,167.4833,78.57,78,0,5.10,NC,Hotel Kou-Bugny
3,Tra Vinh,9.9347,106.3453,71.02,83,0,8.01,VN,Hồng Ân
4,Lunavada,23.1333,73.6167,73.40,23,0,5.75,IN,Arunoday Hotel & Guest House
5,Salalah,17.0151,54.0924,75.20,69,0,2.30,OM,HAMDAN PLAZA HOTEL SALALAH
6,Isangel,-19.5500,169.2667,77.43,81,0,3.40,VU,Volcano Sea View Tree House
7,Voh,-20.9667,164.7000,72.99,86,0,2.55,NC,Le gîte du Koniambo
8,Lima,-12.0432,-77.0282,78.80,61,0,9.22,PE,Gran Hotel Bolivar Lima
9,Luxor,25.6989,32.6421,77.00,31,0,9.28,EG,Iberotel Luxor


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map

hotel_layer = gmaps.marker_layer(
    locations, 
    info_box_content = hotel_info
)

fig.add_layer(hotel_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))